In [1]:
import requests
import json

In [2]:
import pprint
pp = pprint.PrettyPrinter(indent=5)

In addition to the methods using TRAPI, outlined in `HelloRobokop_TRAPI.ipynb` and `HelloRobokop_TRAPI_multiple_IDs.ipynb`, you can use the Cypher Query Language to talk to the graph.  An intro to using the Cypher Query Language can be found here: https://neo4j.com/developer/cypher/guide-cypher-basics/

There are two instances for accessing the graph using cypher.  There is one at http://robokopkg.renci.org which has a cypher browser on it, or you can write cypher and post it there.

The example query below asks "Find me a Gene that is related to both `PUBCHEM.COMPOUND:644073` (Buprenorphine) and `HP:0001337` (Tremor)".

In [8]:
# Buprenorphine -> [Gene] -> Tremor
cypher = f"""MATCH (n0_0:`biolink:ChemicalEntity`)-[r0_0]-(n1_0:`biolink:Gene`)-[r1_0]-(n2_0:`biolink:DiseaseOrPhenotypicFeature`) 
WHERE n0_0.name IN ['Buprenorphine'] AND n2_0.name IN ['Tremor'] 
RETURN [startNode(r0_0),[type(r0_0),properties(r0_0)],endNode(r0_0)] as edge_1,  
[startNode(r1_0),[type(r1_0),properties(r1_0)],endNode(r1_0)] as edge_2, 
[n0_0.name, n1_0.name, n2_0.name] as node_names LIMIT 100"""


The `MATCH` portion tells the structure of the query and names each node and edge. The first node is designated `n0_0` and has the type `biolink:ChemicalEntity`, which is a defined property used in the ROBOKOP KG.  Similarly, the second node is designated `n1_0` with the type `biolink:Gene`, and the third node is designated `n2_0` with the type `biolink:DiseaseOrPhenotypicFeature`.  Between the nodes are the edges, designated `r0_0` for the first edge and `r1_0` for the second edge.

The `WHERE` piece restricts the answers returned to those where the name for `n0_0` is `Buprenorphine`  and the name for `n2_0` is `Tremor`. As currently written, this query could be expanded to allow more than one name for either `n0_0` or `n2_0` by adding additional names to the lists defined by the `[ ]` notation in the query. For example, the query could be expanded to include an additional name for `tremor` as follows: `n2_0.name IN ['Tremor', 'Asterixis']`.

Three query results are returned in this example in the `RETURN` portion.  The first contains the subject and object nodes and the edge properties of the first edge as `edge_1`.  The second contains the subject and object nodes and the edge properties of the second edge as `edge_2`.  The third contains each of the nodes and their names as `node_names`.

Any adjustments made to the `RETURN` section will require further adjustments downstream when extracting results in the sections below.

## Cypher - robokopkg.renci.org

The first instance for accessing the graph using cypher is here using the bolt protocol.  After defining the Neo4jConnection class, the query is sent to http://robokopkg.renci.org

In [9]:
from neo4j import GraphDatabase
class Neo4jConnection:
    
    def __init__(self, uri, user, pwd):
        self.__uri = uri
        self.__user = user
        self.__pwd = pwd
        self.__driver = None
        try:
            self.__driver = GraphDatabase.driver(self.__uri, auth=(self.__user, self.__pwd))
        except Exception as e:
            print("Failed to create the driver:", e)
        
    def close(self):
        if self.__driver is not None:
            self.__driver.close()
        
    def query(self, query, db=None):
        assert self.__driver is not None, "Driver not initialized!"
        session = None
        response = None
        try: 
            session = self.__driver.session(database=db) if db is not None else self.__driver.session()
            response = list(session.run(query))
        except Exception as e:
            print("Query failed:", e)
        finally: 
            if session is not None:
                session.close()
        return response

In [10]:
pw = ''
conn = Neo4jConnection(uri="bolt://robokopkg.renci.org:7687", user = 'neo4j', pwd = pw)
record_list = conn.query(cypher)

In [11]:
from datetime import datetime
from pathlib import Path

now = datetime.now()
dt_string = now.strftime("%Y-%m-%d_%H%M%S")
write_dir = Path("output/Cypher_robokopkg",str(dt_string))
write_dir.mkdir(parents=True, exist_ok=True)


A list of records is returned from the query. The structure of each record is defined by the `RETURN` section of the query above.
`<Record edge_1=[<Node element containing properties for first node of r0_0>,
                 [list containing the type and properties for the edge],
                 <Node element containing properties for second node of r0_0>]
         edge_2=[<Node element containing properties for first node of r1_0>,
                 [list containing the type and properties for the edge],
                 <Node element containing properties for second node of r1_0>]
         node_names=[list of node names]
>

In [17]:
record = record_list[0]
print(record)

<Record edge_1=[<Node element_id='8421444' labels=frozenset({'biolink:NamedThing', 'biolink:PhysicalEssenceOrOccurrent', 'biolink:ChemicalOrDrugOrTreatment', 'biolink:ChemicalEntityOrGeneOrGeneProduct', 'biolink:PhysicalEssence', 'biolink:ChemicalEntity', 'biolink:SmallMolecule', 'biolink:ChemicalEntityOrProteinOrPolypeptide', 'biolink:Entity', 'biolink:MolecularEntity'}) properties={'CHEBI_ROLE_delta_opioid_agent': True, 'smiles': 'CO[C@]12CC[C@@]3(C[C@@H]1[C@](C)(O)C(C)(C)C)[C@H]1CC4=CC=C(O)C5=C4[C@@]3(CCN1CC1CC1)[C@H]2O5', 'description': 'A morphinane alkaloid that is 7,8-dihydromorphine 6-O-methyl ether in which positions 6 and 14 are joined by a -CH2CH2- bridge, one of the hydrogens of the N-methyl group is substituted by cyclopropyl, and a hydrogen at position 7 is substituted by a 2-hydroxy-3,3-dimethylbutan-2-yl group. It is highly effective for the treatment of opioid use disorder and is also increasingly being used in the treatment of chronic pain.', 'fda_labels': 74, 'rgb': 

The data can be accessed using the data() method. Known keys can be passed to data(), but leaving it blank will return everything as a Dictionary. Data in records are returned with keys based on the original query labels. 

In [18]:
record_data = record.data()
pp.pprint(record_data)

{    'edge_1': [    {    'CHEBI_ROLE_agonist': True,
                         'CHEBI_ROLE_analgesic': True,
                         'CHEBI_ROLE_antagonist': True,
                         'CHEBI_ROLE_delta_opioid_agent': True,
                         'CHEBI_ROLE_delta_opioid_receptor_antagonist': True,
                         'CHEBI_ROLE_drug': True,
                         'CHEBI_ROLE_kappa_opioid_agent': True,
                         'CHEBI_ROLE_kappa_opioid_receptor_antagonist': True,
                         'CHEBI_ROLE_mu_opioid_agent': True,
                         'CHEBI_ROLE_mu_opioid_receptor_agonist': True,
                         'CHEBI_ROLE_neurotransmitter_agent': True,
                         'CHEBI_ROLE_opioid_agent': True,
                         'CHEBI_ROLE_opioid_analgesic': True,
                         'CHEBI_ROLE_opioid_receptor_agonist': True,
                         'CHEBI_ROLE_opioid_receptor_antagonist': True,
                         'CHEBI_ROLE_pha

Results are extracted and stored in the format of subject -> predicate -> object, followed by the remaining edge properties. Including the edge properties helps to distinguish edges that may have the same predicates. Unique entries are appended to a list, counted, and then written to a text file.

The code below extracts results based on the structure of the original cypher query in the section above.  Any changes to the `RETURN` part of the query will require adjustments to the code below.

In [22]:
import os
from collections import Counter

#i = 0

string_out_list = []
for record in record_list: 
    record_data = record.data()
    #j = 0
    #only grab the edge information and skip the list of node names
    record_data_first2 = {k: record_data[k] for k in list(record_data)[:2]}
    for label, data in record_data_first2.items():
        string_out = f"{label} - {data[0]['name']} -> {data[1][0]} -> {data[2]['name']}||{data[1][1]}"
        # print(f"{data[0]['name']} -> {data[1][0]} -> {data[2]['name']}")
        # print(f"Edge properties: {data[1][1]}")
        if string_out not in string_out_list:
            string_out_list.append(string_out)
        #j = j + 1
    # print("")

    #i = i + 1
    # if i > 0:
        # break

combined_node_list = "_".join(list(record_list[0].data('node_names').values())[0])
print(combined_node_list)

string_out_list = [i.split('||', 1)[0] for i in string_out_list]

string_out_dict = dict(Counter(string_out_list).items())
pp.pprint(string_out_dict)

with open(os.path.join(write_dir,combined_node_list+".txt"), 'w') as convert_file:
    convert_file.write(json.dumps(string_out_dict))

Buprenorphine_CYP2D6_Tremor
{    'edge_1 - Buprenorphine -> biolink:affects -> CYP2D6': 1,
     'edge_1 - Buprenorphine -> biolink:directly_physically_interacts_with -> CYP2D6': 1,
     'edge_1 - Buprenorphine -> biolink:regulates -> CYP2D6': 1,
     'edge_1 - CYP2D6 -> biolink:affects -> Buprenorphine': 1,
     'edge_2 - CYP2D6 -> biolink:genetic_association -> Tremor': 2}


Close the Cypher connection when finished.

In [25]:
conn.close()

{'query': "MATCH (n0_0:`biolink:ChemicalEntity`)-[r0_0]-(n1_0:`biolink:Gene`)-[r1_0]-(n2_0:`biolink:DiseaseOrPhenotypicFeature`) \nWHERE n0_0.name IN ['Buprenorphine'] AND n2_0.name IN ['Tremor'] \nRETURN [startNode(r0_0),[type(r0_0),properties(r0_0)],endNode(r0_0)] as edge_1,  \n[startNode(r1_0),[type(r1_0),properties(r1_0)],endNode(r1_0)] as edge_2, \n[n0_0.name, n1_0.name, n2_0.name] as node_names LIMIT 100"}

## Cypher - automat.renci.org

The second instance of accessing the graph using cypher is to send through the automat interface (https://automat.renci.org/robokopkg/cypher), using the same cypher query from above, submitted in a json format with an explicitly labeled query field.

The data is returned in a JSON format and can be extracted in a similar way as above after calling the right property of the resulting JSON string. One subtle difference is that the JSON returns each row as a list of ordered columns rather than a format that includes the column labels.

In [34]:
j = {'query': cypher}
results = requests.post('https://automat.renci.org/robokopkg/cypher',json=j)
print(results.status_code)
results_json = results.json()
print(results_json['results'][0]['columns'])
column_names = results_json['results'][0]['columns']

200
['edge_1', 'edge_2', 'node_names']


In [35]:
from datetime import datetime
from pathlib import Path

now = datetime.now()
dt_string = now.strftime("%Y-%m-%d_%H%M%S")
write_dir = Path("output/Cypher_automat",str(dt_string))
write_dir.mkdir(parents=True, exist_ok=True)

In [36]:
pp.pprint(results_json['results'][0]['data'][0]['row'])

[    [    {    'CHEBI_ROLE_agonist': True,
               'CHEBI_ROLE_analgesic': True,
               'CHEBI_ROLE_antagonist': True,
               'CHEBI_ROLE_delta_opioid_agent': True,
               'CHEBI_ROLE_delta_opioid_receptor_antagonist': True,
               'CHEBI_ROLE_drug': True,
               'CHEBI_ROLE_kappa_opioid_agent': True,
               'CHEBI_ROLE_kappa_opioid_receptor_antagonist': True,
               'CHEBI_ROLE_mu_opioid_agent': True,
               'CHEBI_ROLE_mu_opioid_receptor_agonist': True,
               'CHEBI_ROLE_neurotransmitter_agent': True,
               'CHEBI_ROLE_opioid_agent': True,
               'CHEBI_ROLE_opioid_analgesic': True,
               'CHEBI_ROLE_opioid_receptor_agonist': True,
               'CHEBI_ROLE_opioid_receptor_antagonist': True,
               'CHEBI_ROLE_pharmaceutical': True,
               'CHEBI_ROLE_pharmacological_role': True,
               'alogs': -4.44,
               'arom_c': 6,
               'cd_formul

The code below extracts results based on the structure of the original cypher query in the section above.  Any changes to the `RETURN` part of the query will require adjustments to the code below.

In [37]:
import os
from collections import Counter

#i = 0

string_out_list = []
for result in results_json['results'][0]['data']:
    #j = 0
    for item in result['row'][0:2]:
        string_out = f"{item[0]['name']} -> {item[1][0]} -> {item[2]['name']}||{item[1][1]}"
        if string_out not in string_out_list:
            string_out_list.append(string_out)
        #j = j + 1

    #i = i + 1
    # if i > 0:
    #     break
    
combined_node_list = "_".join(results_json['results'][0]['data'][0]['row'][2]).replace(" ", "_")
print(combined_node_list)

string_out_list = [i.split('||', 1)[0] for i in string_out_list]

string_out_dict = dict(Counter(string_out_list).items())
pp.pprint(string_out_dict)

with open(os.path.join(write_dir,combined_node_list+".txt"), 'w') as convert_file:
    convert_file.write(json.dumps(string_out_dict))

Buprenorphine_CYP2D6_Tremor
{    'Buprenorphine -> biolink:affects -> CYP2D6': 1,
     'Buprenorphine -> biolink:directly_physically_interacts_with -> CYP2D6': 1,
     'Buprenorphine -> biolink:regulates -> CYP2D6': 1,
     'CYP2D6 -> biolink:affects -> Buprenorphine': 1,
     'CYP2D6 -> biolink:genetic_association -> Tremor': 2}


## ExEmPLAR

ExEmPLAR (https://www.exemplar.mml.unc.edu/) is an additional GUI-based tool that uses Cypher to explore the ROBOKOP KG. More information on the use of this tool is available here: https://github.com/beasleyjonm/AOP-COP-Path-Extractor. Start and End Nodes can be specified by type and have multiple terms specified.  The labels applied to the Start and End Nodes can be checked against the KG for presence or absence.  Additionally, the number of nodes in-between can be specified along with edge types.  By default, a query from ExEmPLAR will return a collapsed version of edge results. This behavior can be changed by clicking the "Get Result Metadata" checkbox. Note that directionality is not preserved in the query results, but the orientation of the nodes are preserved.  For example, `Buprenorphine` - `CYP2D6` - `Tremor` will be returned with edge labels, but not which direction the edge applies.

Using the same `Buprenorphine` - `Gene` - `Tremor` example as above, three rows are returned.  These are in a collapsed form of the results from queries to `robokopkg.renci.org` above, summarized below.
 - `Buprenorphine` - `affects` - `CYP2D6`
 - `Buprenorphine` - `directly physically interacts with` - `CYP2D6`
 - `Buprenorphine` - `regulates` - `CYP2D6`
 - `CYP2D6` - `genetic association` - `Tremor`

Additionally, cypher queries can be generated by clicking the `Copy` button from the ExEmPLAR tool after setting up a query pattern.  An example using the copied pre-genereated cypher query to each of `automat.renci.org` and `robokopkg.renci.org` is below.

The difficulty faced with the copied query is that "RETURN \*" is not specific with the structures given back.  In the queries to `automat.renci.org` and `robokopkg.renci.org`, while the nodes and their respective information are preserved, the edges have differing amounts of information returned.  The query to `automat.renci.org` gets most of the edge properties except for the edge predicate and subject/object nodes, thus directionality.  The query to `robokopkg.renci.org` returns the opposite, where the edge predicate is solely returned with no edge properties, which makes the returned information appear redundant.

In [38]:
cypher_exemplar = "MATCH (n0_0:`biolink:ChemicalEntity`)-[r0_0]-(n1_0:`biolink:Gene`)-[r1_0]-(n2_0:`biolink:DiseaseOrPhenotypicFeature`) WHERE n0_0.name IN ['Buprenorphine'] AND n2_0.name IN ['Tremor'] RETURN * LIMIT 100"

In [39]:
import pprint
pp = pprint.PrettyPrinter(indent=5)

In [40]:
import requests
import json

j = {'query': cypher_exemplar}
results = requests.post('https://automat.renci.org/robokopkg/cypher',json=j)
results_json = results.json()

In [42]:
print("Query to automat.renci.org")
i = 0
for result in results_json['results'][0]['data']:
    #j = 0
    i = i + 1
    n1 = result['row'][0].get('name') or "NOT FOUND"
    n2 = result['row'][1].get('name') or "NOT FOUND"
    n3 = result['row'][2].get('name') or "NOT FOUND"
    e1 = result['row'][3].get('qualified_predicate') or "NOT FOUND"
    e2 = result['row'][4].get('qualified_predicate') or "NOT FOUND"
    print(f"Result {i}")
    print(f"{n1} -> {e1} -> {n2} -> {e2} -> {n3}")
    print(f"Edge 1 info: {result['row'][3]}")
    print(f"Edge 2 info: {result['row'][4]}\n")
    

Query to automat.renci.org
Result 1
Buprenorphine -> biolink:causes -> CYP2D6 -> NOT FOUND -> Tremor
Edge 1 info: {'object_direction_qualifier': 'decreased', 'qualified_predicate': 'biolink:causes', 'biolink:primary_knowledge_source': 'infores:ctd', 'description': 'decreases activity of', 'NCBITaxon': '9606', 'object_aspect_qualifier': 'activity', 'publications': ['PMID:12756210']}
Edge 2 info: {'score': 0.3, 'biolink:primary_knowledge_source': 'infores:disgenet', 'biolink:aggregator_knowledge_source': ['infores:pharos']}

Result 2
Buprenorphine -> biolink:causes -> CYP2D6 -> NOT FOUND -> Tremor
Edge 1 info: {'object_direction_qualifier': 'increased', 'qualified_predicate': 'biolink:causes', 'biolink:primary_knowledge_source': 'infores:ctd', 'description': 'increases metabolic processing of', 'NCBITaxon': '9606', 'object_aspect_qualifier': 'metabolic_processing', 'publications': ['PMID:12756210']}
Edge 2 info: {'score': 0.3, 'biolink:primary_knowledge_source': 'infores:disgenet', 'biol

In [43]:
pw = ''
conn = Neo4jConnection(uri="bolt://robokopkg.renci.org:7687", user = 'neo4j', pwd = pw)
record_list = conn.query(cypher_exemplar)

In [44]:
import os

print("Query to robokopkg.renci.org")

i = 0
for record in record_list: 
    i = i + 1
    record_data = record.data()
    
    print(f"Result {i}:")
    for label, data in record_data.items():
        if 'r' in label:
            print(f"{data[0]['name']} - {data[1]} - {data[2]['name']}")
    print()
    # if i > 0:
        # break

Query to robokopkg.renci.org
Result 1:
Buprenorphine - biolink:affects - CYP2D6
CYP2D6 - biolink:genetic_association - Tremor

Result 2:
CYP2D6 - biolink:affects - Buprenorphine
CYP2D6 - biolink:genetic_association - Tremor

Result 3:
Buprenorphine - biolink:regulates - CYP2D6
CYP2D6 - biolink:genetic_association - Tremor

Result 4:
Buprenorphine - biolink:directly_physically_interacts_with - CYP2D6
CYP2D6 - biolink:genetic_association - Tremor

Result 5:
Buprenorphine - biolink:affects - CYP2D6
CYP2D6 - biolink:genetic_association - Tremor

Result 6:
CYP2D6 - biolink:affects - Buprenorphine
CYP2D6 - biolink:genetic_association - Tremor

Result 7:
Buprenorphine - biolink:regulates - CYP2D6
CYP2D6 - biolink:genetic_association - Tremor

Result 8:
Buprenorphine - biolink:directly_physically_interacts_with - CYP2D6
CYP2D6 - biolink:genetic_association - Tremor



To summarize, we encountered issues with using the notation "RETURN \*", mainly because this is not specific with what to return.  Queries to `automat` return edge properties, not including direction or predicates, but queries to `robokopkg.renci.org` return the direction and predicates, not including edge properties.  Above, for `robokopkg.renci.org`, it appears as if results 1 & 5, 2 & 6, 3 & 7, and 4 & 8 are duplicates of each other, while for `automat.renci.org`, edge predicates are missing.  We replaced the "\*" with a format to get node pairs and specific relationship information, including the type and properties.  This is demonstrated in the sections [Cypher - robokopkg.renci.org](#Cypher---robokopkg.renci.org) and [Cypher - automat.renci.org](#Cypher---automat.renci.org)